# Import

#### "__future__"
파이썬 2와 3의 버전 차이로 인해 생기는 문제를 방지하고 호환이 되도록 하기 위해 사용합니다.

파이썬 2에서 3의 기능을 가져와서 사용할 수 있게 해줍니다.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
import numpy as np

In [3]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf

#### Tensorflow_hub
텐서플로우 허브(TensorFlow Hub)는 일반화된 문제들에 대해서 모델의 재사용성을 극대화하기 위해서 구글에서 새로 공개한 API이다.

In [0]:
import tensorflow_hub as hub

#### Tensorflow_datasets
텐서플로에서 예제 데이터는 tensorflow_datasets라는 별도의 패키지로 분리되어 있다.

In [0]:
import tensorflow_datasets as tfds

In [7]:
print("버전: ", tf.__version__)

버전:  2.1.0


In [8]:
print("즉시 실행 모드: ", tf.executing_eagerly())

즉시 실행 모드:  True


In [9]:
print("허브 버전: ", hub.__version__)

허브 버전:  0.7.0


In [11]:
print("GPU ", "사용가능" if tf.config.experimental.list_physical_devices("GPU") else "사용 불가!")

GPU  사용가능


## IMDB 데이터셋 다운로드

In [0]:
# 결국 훈련에 15,000개 샘플, 검증에 10,000개 샘플, 테스트에 25,000개 샘플을 사용하게 됩니다.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

## 데이터 탐색

In [0]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [16]:
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [17]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## 모델 구성

In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

In [0]:
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

In [20]:
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [21]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 모델 훈련

In [23]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 6s 211ms/step - loss: 1.7542 - accuracy: 0.5024 - val_loss: 0.8012 - val_accuracy: 0.5167
Epoch 2/20
30/30 [==============================] - 5s 183ms/step - loss: 0.7489 - accuracy: 0.5172 - val_loss: 0.7008 - val_accuracy: 0.5598
Epoch 3/20
30/30 [==============================] - 6s 185ms/step - loss: 0.6661 - accuracy: 0.6028 - val_loss: 0.6412 - val_accuracy: 0.6330
Epoch 4/20
30/30 [==============================] - 6s 185ms/step - loss: 0.6150 - accuracy: 0.6652 - val_loss: 0.6056 - val_accuracy: 0.6748
Epoch 5/20
30/30 [==============================] - 6s 188ms/step - loss: 0.5820 - accuracy: 0.6990 - val_loss: 0.5768 - val_accuracy: 0.7069
Epoch 6/20
30/30 [==============================] - 6s 190ms/step - loss: 0.5530 - accuracy: 0.7249 - val_loss: 0.5506 - val_accuracy: 0.7292
Epoch 7/20
30/30 [==============================] - 6s 190ms/step - loss: 0.5215 - accuracy: 0.7502 - val_loss: 0.5262 - val_accuracy: 0.7444
Epoch 

In [24]:
results = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.326
accuracy: 0.862
